In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
train = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
test = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")

In [ ]:
train.head()

In [ ]:
X_train = train["comment_text"][0:10000]
X_test = test["comment_text"][0:10000]

y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]][0:10000]


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

In [ ]:
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    doc = nlp(text)
    tokens = [word.text for word in doc
            if not word.is_stop
                and not word.is_space
                and not word.is_digit
                and not word.is_punct]
    return ' '.join(tokens)

X_train = X_train.apply(lambda x: preprocess(x))
X_test = X_test.apply(lambda x: preprocess(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(use_idf=True)

train_vectors = tfidf_vectorizer.fit_transform(X_train).toarray()
test_vectors = tfidf_vectorizer.transform(X_test).toarray()

In [ ]:
train_vectors.shape

In [ ]:
from tensorflow.keras import models, layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(35832,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(6, activation='sigmoid'))
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

history = model.fit(train_vectors,
                    y_train,
                    epochs=20,
                    batch_size=128,
                    )

In [ ]:
y_pred = model.predict(test_vectors)

In [ ]:
submission = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')[0:10000]
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('submission.csv', index=False)